In [25]:
import config
import json
import requests
import vulture_scraper as vulture
api_key = config.API_key
import time 

In [3]:
## returns data from Movie DB's list of popular movies. updated daily 
def get_popular(api_key):

    url = "https://api.themoviedb.org/3/movie/popular?api_key={}&language=en-US".format(api_key)
    return requests.get(url).json()

In [4]:
## returns only the titles from Movie DB's list of popular movies 
def get_popular_titles(api_key):
    data = get_popular(api_key)
    return [x['title'] for x in data['results']]

get_popular_titles(api_key)[:5]

['Joker',
 'Maleficent: Mistress of Evil',
 'Terminator: Dark Fate',
 'Fast & Furious Presents: Hobbs & Shaw',
 'The Lion King']

In [5]:
## returns data from Movie DB's list of top rated movies 
def get_top_rated(api_key):
    url = "https://api.themoviedb.org/3/movie/top_rated?api_key={}&language=en-US".format(api_key)
    return requests.get(url).json()

In [6]:
## returns list of IDs from Movie DB's top rated movies 
def get_top_rated_ids(api_key):
    data = get_top_rated(api_key)
    return [x['id'] for x in data['results']]


In [7]:
## returns full details when given a list of movie IDs
def get_movies(ids):
    data = []
    for i in ids:
        url = "https://api.themoviedb.org/3/movie/{}?api_key={}&language=en-US".format(i,api_key)
        data.append(requests.get(url).json())
    return data

In [26]:
## returns movie data when given an API key and list of movie titles 

def search_movies(api_key, list_of_movies):
    data = []
    for movie in list_of_movies[0:40]:
        url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
        data.append(requests.get(url).json())
        time.sleep(1)
    time.sleep(10)
    for movie in list_of_movies[41:]:
        url = "https://api.themoviedb.org/3/search/movie?api_key={}&language=en-US&query={}".format(api_key,movie)
        data.append(requests.get(url).json())
        time.sleep(1)
    return data


In [27]:
## search Movie DB for the best movies on Hulu according to Vulture 

hulu_movies = search_movies(api_key, vulture_scraper.best_hulu)

In [35]:
netflix_movies = search_movies(api_key, vulture.best_netflix)

In [36]:
amazon_movies = search_movies(api_key, vulture.best_amazon)

In [50]:
import pandas as pd

In [60]:
hulu_results = [x['results'] for x in hulu_movies]
for x in hulu_results:
    for y in x:
        print(y['id'])
        

814
157438
61997
829
23298
38469
38788
190238
531820
583391
59701
113751
297048
279708
305571
601940
475299
275868
409058
282774
156399
71702
247957
447332
520763
384154
473942
105079
592618
92927
7191
333371
384521
608751
603
604
605
14543
624860
221495
344225
503880
591955
505806
274866
371610
174615
26214
119742
408912
323256
268477
472854
55931
489988
389015
813
2665
550115
258210
501309
544233
55034
587854
429808
588172
587852
454332
578038
316432
633033
179828
575113
136469
14052
16889
21029
18543
44264
17529
108807
599745
589911
222190
9532
19912
55779
9286
9358
42819
574475
552763
106141
428889
469369
251721
7326
47695
72248
159361
322952
342481
504406
482106
177947
342480
342482
63210
465994
101820
427386
611411
543005
173720
303407
403403
9003
8766
12597
444149
12699
9064
11246
70584
17455
11569
338947
348191
377998
365379
520103
555324
550716
550718
404370
482264
14154
437511
581797
590438
548168
78352
211917
412423
860
2640
419326
465914
592998
957
56297
15067
10585
533642
